In [3]:
import pandas as pd  # Data handling
from scipy.integrate import odeint # numerical integration
import numpy as np
from pyvbmc import VBMC # VMBC object
from pyvbmc import VariationalPosterior
from pyvbmc.priors import SplineTrapezoidal
from scipy.optimize import minimize
import scipy.stats as scs
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from pyvbmc import VariationalPosterior

def derivs(t,curr_vals,a,b):
    x,xd = curr_vals
    xdd = -x - np.sign(x) * a * np.abs(x)**b
    return  xd,xdd

# time array for solution
tmax = 10
dt = 0.05
t = np.arange(0,tmax,dt)

# Range of values that parameters can take on
a_min = 0.0
a_max = 0.5
b_min = 1.0
b_max = 5.0

# Number of parameters (dimension)
D = 2

# Bounds for VBMC object (LB and UB expanded a bit beyond what the actual true values could be)
LB = np.full((1, D), a_min)
LB[0][0] = a_min
LB[0][1] = b_min - 1
UB = np.full((1, D), a_max)
UB[0][0] = a_max + 0.1
UB[0][1] = b_max + 1
PLB = np.copy(LB)
PLB[0][0] = a_min + 0.01
PLB[0][1] = b_min
PUB = np.copy(UB)
PUB[0][0] = a_max 
PUB[0][1] = b_max

#set up prior
prior = SplineTrapezoidal(LB, PLB, PUB, UB)
options = {
    "display": "off"
}

print("LB:",LB,"PLB:",PLB,"PUB:",PUB,"UB:",UB,'x0_rand:',x0_rand)

def log_likelihood(theta):
    a,b = theta
    sols = []
    for initial_condition in initial_conditions:
        sol = solve_ivp(derivs,(0,tmax),initial_condition,t_eval = t, args = (a,b), method = "LSODA").y
        sols.append(sol[0])
    big_sol = np.concatenate(sols)
    sliced_sol = big_sol[0::4]
    LL =  np.sum(-(0.5/sigma**2) * (y_true - sliced_sol)**2) #+ np.log(1/(sigma* (2*np.pi)**0.5))
    return LL

initial_conditions = [[0.0, 3.5],[2.0, 0.0],[-0.5, -0.5],[3.14, -0.3]]

sigma = 0.1
noise_sigma = 0.05

a = 0.3
b = 2.0
sols = []

N = 7
a_vals = np.linspace(0.1,0.5,N)
b_vals = np.linspace(1.1,3.5,N)
print(a_vals)
print(b_vals)

LB: [[0. 0.]] PLB: [[0.01 1.  ]] PUB: [[0.5 5. ]] UB: [[0.6 6. ]] x0_rand: [[0.48384462 3.188929  ]]
[0.1        0.16666667 0.23333333 0.3        0.36666667 0.43333333
 0.5       ]
[1.1 1.5 1.9 2.3 2.7 3.1 3.5]


In [5]:
seeds = np.array([0,1]) # CHANGE THIS
count = 0
for seed in seeds:
    
    for i,a in enumerate(a_vals):

        for j,b in enumerate(b_vals):
            sols = []
            for initial_condition in initial_conditions:
                sol = solve_ivp(derivs,(0,tmax),initial_condition,t_eval = t, args = (a,b),method = "LSODA").y
                sols.append(sol[0])
            big_sol = np.concatenate(sols)
            sliced = big_sol[0::4]
            np.random.seed(seed)
            noisy_sliced_sol = sliced + noise_sigma * np.random.randn(len(sliced))
            y_true = noisy_sliced_sol
            np.random.seed(seed)
            count += 1
            np.random.seed(seed)
            x0 = np.random.uniform(PLB, PUB)
            np.random.seed(seed)
            vbmc = VBMC(log_likelihood, x0, LB, UB, PLB, PUB, prior = prior, options = options)
            np.random.seed(seed)
            vp, results = vbmc.optimize()
            vp.save("vp" + str(i) + "_" +  str(j) + "_" + str(seed) + ".hdf5", overwrite=True)
print(count)

Inference terminated: variational solution stable for options.tol_stable_count fcn evaluations.
Estimated ELBO: -34.086 +/-0.110.
Inference terminated: variational solution stable for options.tol_stable_count fcn evaluations.
Estimated ELBO: -34.670 +/-0.013.
Inference terminated: reached maximum number of function evaluations options.max_fun_evals.
Estimated ELBO: -34.745 +/-0.009.
Caution: Returned variational solution may have not converged.
Inference terminated: variational solution stable for options.tol_stable_count fcn evaluations.
Estimated ELBO: -35.270 +/-0.060.
Inference terminated: reached maximum number of function evaluations options.max_fun_evals.
Estimated ELBO: -36.230 +/-0.011.
Caution: Returned variational solution may have not converged.
Inference terminated: reached maximum number of function evaluations options.max_fun_evals.
Estimated ELBO: -36.968 +/-0.011.
Caution: Returned variational solution may have not converged.
Inference terminated: reached maximum numbe

c:\Users\walki\.conda\envs\bpe\Lib\site-packages\cma\evolution_strategy.py:3823: RuntimeWarning: invalid value encountered in scalar subtract
  current_fitness_range = max(es.fit.fit) - min(es.fit.fit)


Inference terminated: reached maximum number of function evaluations options.max_fun_evals.
Estimated ELBO: -1890.668 +/-1.621.
Caution: Returned variational solution may have not converged.
Inference terminated: reached maximum number of function evaluations options.max_fun_evals.
Estimated ELBO: -44.085 +/-1.635.
Caution: Returned variational solution may have not converged.
Inference terminated: reached maximum number of function evaluations options.max_fun_evals.
Estimated ELBO: -46.205 +/-0.079.
Caution: Returned variational solution may have not converged.
Inference terminated: reached maximum number of function evaluations options.max_fun_evals.
Estimated ELBO: -109.342 +/-10.920.
Caution: Returned variational solution may have not converged.
Inference terminated: reached maximum number of function evaluations options.max_fun_evals.
Estimated ELBO: -35.267 +/-0.016.
Caution: Returned variational solution may have not converged.
Inference terminated: reached maximum number of fun

c:\Users\walki\.conda\envs\bpe\Lib\site-packages\cma\evolution_strategy.py:3829: RuntimeWarning: invalid value encountered in scalar subtract
  current_fitness_range < opts['tolfunrel'] * (es.fit.median0 - es.fit.median_min),
c:\Users\walki\.conda\envs\bpe\Lib\site-packages\cma\evolution_strategy.py:3829: RuntimeWarning: invalid value encountered in scalar multiply
  current_fitness_range < opts['tolfunrel'] * (es.fit.median0 - es.fit.median_min),


Inference terminated: reached maximum number of function evaluations options.max_fun_evals.
Estimated ELBO: -146.573 +/-11.190.
Caution: Returned variational solution may have not converged.
Inference terminated: reached maximum number of function evaluations options.max_fun_evals.
Estimated ELBO: -2912.789 +/-79.197.
Caution: Returned variational solution may have not converged.
Inference terminated: variational solution stable for options.tol_stable_count fcn evaluations.
Estimated ELBO: -37.159 +/-0.041.
Inference terminated: reached maximum number of function evaluations options.max_fun_evals.
Estimated ELBO: -39.028 +/-0.153.
Caution: Returned variational solution may have not converged.
Inference terminated: reached maximum number of function evaluations options.max_fun_evals.
Estimated ELBO: -40.394 +/-1.576.
Caution: Returned variational solution may have not converged.
Inference terminated: reached maximum number of function evaluations options.max_fun_evals.
Estimated ELBO: -